Parallel Maps Implementations are Common
---------------------------

Almost every parallel computing framework implements an embarrassingly parallel map (applying the same function to many inputs.)  We have *many* options.  

They all perform about the same.

This notebook goes through the user interface of several of them on the same problem.

In [ ]:
from glob import glob
import ujson as json
import pandas as pd

In [ ]:
filenames = sorted(glob(os.path.join('..', 'data', 'json', '*.json')))  # ../data/json/*.json

def load_parse_store(fn):
    with open(fn) as f:
        data = [json.loads(line) for line in f]

    df = pd.DataFrame(data)

    out_filename = fn[:-5] + '.h5'
    df.to_hdf(out_filename, '/data')



### Sequential for loops

In [ ]:
%%time

for fn in filenames:
    load_parse_store(fn)

### Concurrent.futures

In [ ]:
%%time

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor(4)

list(e.map(load_parse_store, filenames))

### Multiprocessing

In [ ]:
%%time 

from multiprocessing import Pool
p = Pool(4)

list(p.map(load_parse_store, filenames))

### Joblib

In [ ]:
%%time 

from joblib import Parallel, delayed

result = Parallel(n_jobs=4, backend='multiprocessing')(delayed(load_parse_store)(fn) for fn in filenames)

### IPython Parallel

Start an IPython cluster with:

    ipcluster start

In [ ]:
from subprocess import Popen
ipcluster = Popen(['ipcluster', 'start', '-n', '4'])

In [ ]:
import ipyparallel as ipp
c = ipp.Client()

# use the same serialization as everyone else
c[:].use_cloudpickle()

view = c.load_balanced_view()

In [ ]:
%%time

result = list(view.map(load_parse_store, filenames))

In [ ]:
!ipcluster stop

### PySpark

In [ ]:
%%time

import pyspark

sc = pyspark.SparkContext('local[4]')

In [ ]:
%%time

rdd = sc.parallelize(filenames)
result = rdd.map(load_parse_store).collect()

### Dask.bag

In [ ]:
%%time

import dask.bag as db

b = db.from_sequence(filenames)
b.map(load_parse_store).compute()

### Dask.delayed

In [ ]:
%%time

from dask import delayed, compute
import dask.multiprocessing

delayed_values = [delayed(load_parse_store)(fn) for fn in filenames]

compute(*delayed_values, get=dask.multiprocessing.get)

### Dask.distributed

In [ ]:
%%time
from distributed import Executor
e = Executor()  # creates local scheduler and workers

In [ ]:
%%time

futures = e.map(load_parse_store, filenames)
e.gather(futures)